In [3]:
import cv2
import mediapipe as mp
import numpy as np
import os
import pickle

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.5)

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    keypoints = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        if results.pose_landmarks:
            keypoints.append(np.array([[lmk.x, lmk.y] for lmk in results.pose_landmarks.landmark]).flatten())

    cap.release()
    return np.mean(keypoints, axis=0) if keypoints else None

def extract_keypoints_from_videos(videos_folder, label):
    keypoints = []
    labels = []

    for video_file in os.listdir(videos_folder):
        video_path = os.path.join(videos_folder, video_file)
        video_keypoints = process_video(video_path)

        if video_keypoints is not None:
            keypoints.append(video_keypoints)
            labels.append(label)

    return np.array(keypoints), np.array(labels)

# Paths to your video folders
correct_videos_folder = './Correct sequence'
incorrect_videos_folder = './Wrong sequence'

# Extract keypoints
correct_keypoints, correct_labels = extract_keypoints_from_videos(correct_videos_folder, 1)  # Label 1 for correct
incorrect_keypoints, incorrect_labels = extract_keypoints_from_videos(incorrect_videos_folder, 0)  # Label 0 for incorrect

# Combine data
all_keypoints = np.concatenate([correct_keypoints, incorrect_keypoints])
all_labels = np.concatenate([correct_labels, incorrect_labels])

# Save the data
with open('keypoints_data.pkl', 'wb') as f:
    pickle.dump((all_keypoints, all_labels), f)


In [4]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load the preprocessed data
with open('keypoints_data.pkl', 'rb') as f:
    all_keypoints, all_labels = pickle.load(f)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_keypoints, all_labels, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Evaluate the model
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

# Save the trained model
with open('trained_model.pkl', 'wb') as f:
    pickle.dump(model, f)


              precision    recall  f1-score   support

           0       1.00      0.75      0.86         8
           1       0.86      1.00      0.92        12

    accuracy                           0.90        20
   macro avg       0.93      0.88      0.89        20
weighted avg       0.91      0.90      0.90        20



In [5]:
import cv2
import mediapipe as mp
import numpy as np
import pickle

# Load the trained model
with open('trained_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.5)

def draw_landmarks(frame, landmarks):
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing.draw_landmarks(frame, landmarks, mp_pose.POSE_CONNECTIONS)

def process_and_predict(video_path, model, pose):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        if results.pose_landmarks:
            # Draw keypoints
            draw_landmarks(frame, results.pose_landmarks)

            # Predict
            keypoints = np.array([[lmk.x, lmk.y] for lmk in results.pose_landmarks.landmark]).flatten()
            avg_keypoints = keypoints.reshape(1, -1)
            prediction = model.predict(avg_keypoints)
            text = "Correct" if prediction[0] == 1 else "Incorrect"

            # Display prediction
            cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow('Pushup Analysis', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Example usage
video_path = 'P:/PoseDetector/Push ups/wrong2.mp4'
process_and_predict(video_path, loaded_model, pose)


In [7]:
def real_time_predict(model, pose):
    cap = cv2.VideoCapture(0)  # 0 for the default webcam
    print("Press 'q' to quit")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        if results.pose_landmarks:
            keypoints = np.array([[lmk.x, lmk.y] for lmk in results.pose_landmarks.landmark]).flatten()
            avg_keypoints = keypoints.reshape(1, -1)
            prediction = model.predict(avg_keypoints)
            text = "Correct" if prediction[0] == 1 else "Incorrect"
            cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow('Pushup Analysis', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Example usage for real-time prediction
real_time_predict(loaded_model, pose)


Press 'q' to quit
